## Load data

In [2]:
import pandas as pd

In [17]:
import numpy as np

In [3]:
df_legit = pd.read_csv('./data/all_legit.txt', sep=' ', names=['domain', 'label'])
df_dga = pd.read_csv('./data/all_dga.txt', sep=' ', names=['domain', 'label'])

In [4]:
df_data = pd.concat([df_legit, df_dga])

In [5]:
# Let's ignore the different DGA sources for now. Group into 2 classes: legit or dga
df_data['label'] = df_data.apply(lambda x: x.label > 0, axis=1)

In [6]:
print df_legit.shape
print df_dga.shape
print df_data.shape

(1000000, 2)
(801667, 2)
(1801667, 2)


## Build sklearn model

In [9]:
import math
from collections import Counter
 
def entropy(s):
    p, lns = Counter(s), float(len(s))
    return -sum( count/lns * math.log(count/lns, 2) for count in p.values())

In [12]:
df_data['f_entropy'] = df_data.domain.apply(entropy)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
alexa_vc = CountVectorizer(analyzer='char', ngram_range=(3,5), min_df=1e-4, max_df=1.0)
counts_matrix = alexa_vc.fit_transform(df_data['domain'])
alexa_counts = np.log10(counts_matrix.sum(axis=0).getA1())

In [21]:
df_wordlist = pd.read_csv('data/words.txt', names=['word'], header=None, dtype={'word': np.str}, encoding='utf-8')

df_wordlist = df_wordlist[df_wordlist['word'].map(lambda x: str(x).isalpha())]
df_wordlist = df_wordlist.applymap(lambda x: str(x).strip().lower())
df_wordlist = df_wordlist.dropna()
df_wordlist = df_wordlist.drop_duplicates()

In [24]:
words_vc = CountVectorizer(analyzer='char', ngram_range=(3,5), min_df=1e-5, max_df=1.0)
counts_matrix = words_vc.fit_transform(df_wordlist['word'])
words_counts = np.log10(counts_matrix.sum(axis=0).getA1())

In [26]:
# Compute NGram matches for all the domains and add to our dataframe
df_data['f_alexa_grams'] = alexa_counts * alexa_vc.transform(df_data['domain']).T 
df_data['f_word_grams'] = words_counts * words_vc.transform(df_data['domain']).T 

df_data['f_diff'] = df_data['f_alexa_grams'] - df_data['f_word_grams']

In [27]:
from sklearn.model_selection import train_test_split

# Create training and test split
X_train, X_test = train_test_split(df_data, test_size = .2)

In [28]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=20)

In [31]:
features = ['f_entropy', 'f_alexa_grams', 'f_word_grams', 'diff']
clf.fit(X_train[features], X_train.label)
y_pred = clf.predict(X_test[features])

In [34]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
print precision_recall_fscore_support(X_test.label, y_pred)
confusion_matrix(X_test.label, y_pred > .5)

(array([ 0.93215841,  0.91385821]), array([ 0.93054317,  0.91582636]), array([ 0.93135009,  0.91484123]), array([199678, 160656]))


array([[185809,  13869],
       [ 13523, 147133]])

## Build LSTM model

In [6]:
from sklearn.model_selection import train_test_split

In [9]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Quadro K4000 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)


In [7]:
# Tokenise by converting each char into int repr
df_data['domain_char'] = df_data.apply(lambda x: [ord(c) for c in x.domain], axis=1)

In [8]:
# Create training and test split
X_train, X_test = train_test_split(df_data, test_size = .2)

NameError: name 'train_test_split' is not defined

In [10]:
maxlen = 100
x_train = sequence.pad_sequences(X_train.domain_char, maxlen=maxlen, value=-1)
x_test = sequence.pad_sequences(X_test.domain_char, maxlen=maxlen, value=-1)

In [11]:
model = Sequential()
model.add(Embedding(256, 128, input_length=maxlen))
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, X_train.label,
         batch_size=256,
         epochs=5,
         validation_split=.1)

Train on 1297199 samples, validate on 144134 samples
Epoch 1/5
 648192/1297199 [=============>................] - ETA: 813s - loss: 0.1086 - acc: 0.9597

In [ ]:
model.evaluate(x_test, X_test.label)

In [23]:
y_pred = model.predict(x_test, batch_size=256, verbose=1)

360334/360334 [==============================] - 126s   


array([[197905,   2058],
       [  2321, 158050]])

In [30]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
print precision_recall_fscore_support(X_test.label, y_pred > .5)
confusion_matrix(X_test.label, y_pred > .5)

(array([ 0.9884081 ,  0.98714618]), array([ 0.9897081 ,  0.98552731]), array([ 0.98905767,  0.98633608]), array([199963, 160371]))


array([[197905,   2058],
       [  2321, 158050]])

In [31]:
model.save('./dga-bot.h5')